## Some Notes

A note about astropy convolutions, ``convolve()`` is better for small kernels, ``convolve_fft()`` is better for larger kernels, http://docs.astropy.org/en/stable/convolution/, http://docs.astropy.org/en/stable/convolution/using.html.  We will use ``convolve`` for the examples in this notebook.  You can select the following boundary rules: none, fill, wrap, and extend (?). Convolution always done as floating point.  Check out list of kernels @ astropy,scipy,stsci one

** Note about extra astropy features, but somethings only in scipy **

## General Imports
These imports will be used for a majority of the tasks listed below.  More specific imports can be found in the task section.

In [31]:
from astropy.io import fits
import numpy as np
from astropy.convolution import ap_convolve
from scipy.ndimage import convolve as sp_convolve

## Imports for Plotting Examples

In [102]:
# imports...

<center> <h2>Convolution Functions</h2> </center>

### convolve

The convolve task allows you to convolve your data array with a kernel of your own creation.  Here we show a simple example of a rectangular kernel applied to a 10 by 10 array using the ``astropy.convolution.convolve`` function

In [19]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# setup our custom kernel
my_kernel = [[0,1,0],[1,0,1],[0,1,0],[1,0,1],[0,1,0]]
# perform convolution
result = convolve(my_arr, my_kernel, normalize_kernel=True, boundary='wrap')

### boxcar

The boxcar convolution does a boxcar smoothing with a given box size, and applies this running average to an array.  Here we show a 2-D example using ``Box2DKernel``, which is convient for square box sizes.

In [7]:
from astropy.convolution import Box2DKernel

In [18]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# setup our kernel
box_kernel = Box2DKernel(3)
# perform convolution
result = convolve(my_arr, box_kernel, normalize_kernel=True)

Add example of rectangular boxcar / ask astropy why they didn't include this...

### gauss

The gaussian kernel convolution applies a gaussian function convolution to your data array.  The ``Gaussian2DKernel`` size is defined slightly differently from the IRAF version: http://docs.astropy.org/en/stable/api/astropy.convolution.Gaussian2DKernel.html#astropy.convolution.Gaussian2DKernel

In [21]:
from astropy.convolution import Gaussian2DKernel

In [32]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# setup our kernel, with 6 sigma and a 3 in x by 5 in y size
gauss_kernel = Gaussian2DKernel(6, x_size=3, y_size=5)
# perform convolution
result = convolve(my_arr, gauss_kernel, normalize_kernel=True)

### laplace

The laplace task runs a image convolution using a laplacian filter with a subset of footprints.  For the ``scipy.ndimage.filter.laplace`` function we will be using, you can feed any footprint in as an array to create your kernel.

In [28]:
from scipy.ndimage import laplace

In [37]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# setup our laplace kernel with a target footprint (diagonals in IRAF)
footprint = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
laplace_kernel = laplace(footprint)
# perform scipy convolution
result = sconvolve(my_arr, laplace_kernel)

### gradient

In [46]:
from scipy.ndimage.filters import generic_gradient_magnitude
from scipy.misc import derivative
from scipy import exp2

In [56]:
footprint = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
out = generic_gradient_magnitude(footprint,????)

1.99999999995


TypeError: f() takes exactly 1 argument (5 given)

<center> <h2>Convolution Functions With Masking</h2> </center>

In [103]:
# do this

<center> <h2>Filter Functions</h2> </center>

### median / rmedian

Apply a median filter to your data array.  We will use the ``scipy.ndimage.filters.median_filter`` function.

In [71]:
from scipy.ndimage.filters import median_filter

In [73]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# apply median filter
filtered = median_filter(my_arr,size=(3,4))

For a ring median filter we can supply a more specific footprint to the ``median_filter`` function.  You can easily generate this footprint using the ``astroimtools`` library

In [81]:
from astroimtools import circular_annulus_footprint

#depreciation warning, is fixed already in the dev version, not sure when this is getting pushed

In [82]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# create annulus filter
fp = circular_annulus_footprint(10, 12)
# apply median filter
filtered = median_filter(my_arr, footprint=fp)

<center> <h2>Creating Your Own Filter Functions</h2> </center>

###  mode / rmode

The mode calculation equation used in the mode and rmode IRAF tasks (3.0\*median - 2.0\*mean) can be recreated using the ``scipy.ndimage.generic_filter`` function. ** explination for approximation**

In [87]:
from scipy.ndimage import generic_filter
from astroimtools import circular_annulus_footprint

In [86]:
def mode_func(in_arr):
    f = 3.0*np.median(in_arr) - 2.0*np.mean(in_arr)
    return f

For a box footprint:

In [91]:
# create test array
my_arr = np.random.randint(0,10,(10,10))
# apply mode filter
filtered = generic_filter(my_arr,mode_func,size=5)

For a ring footprint:

In [99]:
# create test array
my_arr = np.random.randint(0,10,(20,20))
# create annulus filter
fp = circular_annulus_footprint(5, 9)
# apply mode filter
filtered = generic_filter(my_arr,mode_func,footprint=fp)

## Not Covering

### runmed

Please see **images.imutil.imsum**

### fmedian

runmed seems to be kind of complicated, should figure out if people ever use this...

### fmode